### 🔧 1. Simple Forward Propagation: `J(theta) = θx`


In [1]:
import numpy as np

def forward_propagation(x, theta):
    J = np.dot(theta, x)
    return J

x, theta = 2, 4
J = forward_propagation(x, theta)
print("J =", J)


J = 8


### 🔁 2. Simple Backward Propagation (Gradient of J w.r.t. θ)


In [2]:
def backward_propagation(x, theta):
    dtheta = x
    return dtheta

x, theta = 3, 4
dtheta = backward_propagation(x, theta)
print("dtheta =", dtheta)


dtheta = 3


### 🧪 3. Gradient Check for Scalar Case


In [3]:
def gradient_check(x, theta, epsilon=1e-7, print_msg=False):
    theta_plus = theta + epsilon
    theta_minus = theta - epsilon
    J_plus = forward_propagation(x, theta_plus)
    J_minus = forward_propagation(x, theta_minus)
    gradapprox = (J_plus - J_minus) / (2 * epsilon)
    
    grad = backward_propagation(x, theta)
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator
    
    if print_msg:
        if difference > 2e-7:
            print("\033[93mThere is a mistake! difference =", difference, "\033[0m")
        else:
            print("\033[92mBackward propagation is correct! difference =", difference, "\033[0m")
    
    return difference

x, theta = 3, 4
difference = gradient_check(x, theta, print_msg=True)


Backward propagation is correct! difference = 7.814075313343006e-11 


### ⚙️ 4. Multi-layer Forward Propagation (for neural network)


In [4]:
from gc_utils import sigmoid, relu

def forward_propagation_n(X, Y, parameters):
    m = X.shape[1]
    W1, b1 = parameters["W1"], parameters["b1"]
    W2, b2 = parameters["W2"], parameters["b2"]
    W3, b3 = parameters["W3"], parameters["b3"]
    
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    log_probs = np.multiply(-np.log(A3), Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1. / m * np.sum(log_probs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    return cost, cache


### 🔁 5. Backward Propagation for Multi-layer NN


In [5]:
def backward_propagation_n(X, Y, cache):
    m = X.shape[1]
    Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3 = cache

    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims=True)

    dA2 = np.dot(W3.T, dZ3)
    dZ2 = dA2 * (A2 > 0)
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * (A1 > 0)
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims=True)

    gradients = {
        "dW3": dW3, "db3": db3,
        "dW2": dW2, "db2": db2,
        "dW1": dW1, "db1": db1
    }
    return gradients


### ✅ 6. Gradient Check for Multi-layer NN


In [6]:
from gc_utils import dictionary_to_vector, vector_to_dictionary, gradients_to_vector

def gradient_check_n(parameters, gradients, X, Y, epsilon=1e-7, print_msg=False):
    parameters_values, _ = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    
    num_parameters = parameters_values.shape[0]
    gradapprox = np.zeros((num_parameters, 1))
    
    for i in range(num_parameters):
        theta_plus = np.copy(parameters_values)
        theta_plus[i] += epsilon
        J_plus, _ = forward_propagation_n(X, Y, vector_to_dictionary(theta_plus))

        theta_minus = np.copy(parameters_values)
        theta_minus[i] -= epsilon
        J_minus, _ = forward_propagation_n(X, Y, vector_to_dictionary(theta_minus))

        gradapprox[i] = (J_plus - J_minus) / (2 * epsilon)

    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator

    if print_msg:
        if difference > 2e-7:
            print("\033[93mThere is a mistake in backward propagation! difference =", difference, "\033[0m")
        else:
            print("\033[92mYour backward propagation works correctly! difference =", difference, "\033[0m")

    return difference


### 🧪 Example: Run Gradient Check on a Simple Neural Network


In [7]:
import numpy as np

# --- Mock gradient_check_n_test_case() ---

def gradient_check_n_test_case():
    np.random.seed(1)
    X = np.random.randn(4, 3)         # 4 features, 3 examples
    Y = (np.random.randn(1, 3) > 0)   # binary labels

    W1 = np.random.randn(5, 4)
    b1 = np.random.randn(5, 1)
    W2 = np.random.randn(3, 5)
    b2 = np.random.randn(3, 1)
    W3 = np.random.randn(1, 3)
    b3 = np.random.randn(1, 1)

    parameters = {
        "W1": W1, "b1": b1,
        "W2": W2, "b2": b2,
        "W3": W3, "b3": b3
    }

    return X, Y.astype(int), parameters


In [9]:
# --- Run the test case and gradient check ---
X, Y, parameters = gradient_check_n_test_case()

# Forward + Backward
cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y, 1e-7, True)
expected_values = [0.2850931567761623, 1.1890913024229996e-07]
assert not(type(difference) == np.ndarray), "You are not using np.linalg.norm for numerator or denominator"
assert np.any(np.isclose(difference, expected_values)), "Wrong value. It is not one of the expected values"


Your backward propagation works correctly! difference = 7.229555355926147e-09 


AssertionError: Wrong value. It is not one of the expected values